<a href="https://colab.research.google.com/github/MOHOAzure/AI_Paint_Online/blob/main/Colab/WebUI_My_Models_Collections.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Thanks for your contributions

藏經閣
https://drive.google.com/drive/folders/1vGc16Bb8CDW1piUj_5thzbsCmDFamYyt

In [ ]:
#@title Test
# !pip list

In [ ]:
#@title Setup Tool to use models on My Drive

# (optional: check GPU)
# !nvidia-smi

from pathlib import Path

# param from user
version = 'latest' #@param ["latest", "685f9631b56ff8bd43bce24ff5ce0f9a0e9af490"] {allow-input: true}

# model selection
model_and_misc_dir = '/content/drive/Shareddrives/AI_Paint/stable-diffusion' #@param ["/content/drive/Shareddrives/AI_Paint/stable-diffusion", "/content/drive/MyDrive/stable-diffusion"] {allow-input: true}

## the model dir where the tool can find models
model_dir = "models" #@param {type:"string"}

## specific model under the model dir, including extension
model_filename = "perfectWorld_v2Baked.safetensors" #@param {type:"string"}

use_specific_model = False #@param {type:"boolean"}

use_cuda = True # @param {type:"boolean"}

# define variables such as working directory
root_dir = Path("/content")
tool_dir = root_dir/"stable-diffusion-webui"
model_and_misc_dir = Path(model_and_misc_dir)
model_dir = model_and_misc_dir/model_dir

# load Google Drive for my models
from google.colab import drive
drive.mount('/content/drive')

def use_cuda():
  # use cuda instead of cpu for large model (e.g., 7G)
  !sed -i 's/weight_load_location = None if cmd_opts.lowram else "cpu"/weight_load_location = None if cmd_opts.lowram else "cuda"/g' {tool_dir}/modules/shared.py

def install_dep():
  # install xformers & its dep.
  %cd {root_dir}
  !pip install -q torch==2.0.0+cu118 torchvision==0.15.1+cu118 torchaudio==2.0.1+cu118 torchtext==0.15.1 torchdata==0.6.0 --extra-index-url https://download.pytorch.org/whl/cu118 -U
  !pip install -q xformers==0.0.18 triton==2.0.0 -U
  # !pip install xformers==0.0.18
  
  # !python -m xformers.info

  # old versions
  # !pip install tensorflow==2.9.2
  # !pip install fastapi==0.90.0

def install_extentions():
  %cd {tool_dir}/extensions
  
  # Commonly used extention
  !git clone https://github.com/Coyote-A/ultimate-upscale-for-automatic1111.git
  !git clone https://github.com/hako-mikan/sd-webui-regional-prompter.git
  # !git clone https://github.com/ashen-sensored/stable-diffusion-webui-two-shot.git
  
  # !git clone https://github.com/DominikDoom/a1111-sd-webui-tagcomplete.git
  # !git clone https://github.com/toriato/stable-diffusion-webui-wd14-tagger
  # !git clone https://github.com/ThereforeGames/unprompted.git
  # !git clone https://github.com/hnmr293/sd-webui-cutoff.git
  # !git clone https://github.com/7eu7d7/DreamArtist-sd-webui-extension.git
  # !git clone https://github.com/bbc-mc/sdweb-merge-board
  # !git clone https://github.com/tkalayci71/embedding-inspector
  # !git clone https://github.com/bbc-mc/sdweb-merge-block-weighted-gui
  # !git clone https://github.com/Malisius/booru2prompt.git
  # !git clone https://github.com/adieyal/sd-dynamic-prompts
  # !git clone https://github.com/kousw/stable-diffusion-webui-daam
  # !git clone https://github.com/Akegarasu/sd-webui-model-converter.git
  # !git clone https://github.com/hnmr293/sd-webui-llul.git

%cd $root_dir

# install tool: stable-diffusion-webui
repo = "https://github.com/AUTOMATIC1111/stable-diffusion-webui.git"
!git clone $repo

# time machine: commit or branch
if version != 'latest':
  %cd {tool_dir}
  !git checkout $version

install_dep()
install_extentions()

# load tool config
!cp {model_and_misc_dir}/"[template] ui-config.json" {tool_dir}/ui-config.json
!cp {model_and_misc_dir}/"[template] config.json" {tool_dir}/config.json

if use_specific_model:
  model = model_dir / model_filename

# additional VAE
# !cp /content/drive/MyDrive/stable-diffusion/models/VAE/orangemix.vae.pt {tool_dir}/models/VAE/
# embedding
# !cp /content/drive/MyDrive/stable-diffusion/models/embeddings/* {tool_dir}/embeddings/

if use_cuda:
  use_cuda()

In [ ]:
#@title Install extension: supermerge
use_supermerge = False # @param {type:"boolean"}
if use_supermerge:
  %cd {tool_dir}/extensions  
  !git clone https://github.com/hako-mikan/sd-webui-supermerger.git

In [ ]:
#@title Install extension: controlnet

use_controlnet = False # @param {type:"boolean"}

if use_controlnet:
  # @markdown ---
  # @markdown - Info: ControlNet
  # @markdown  - https://github.com/lllyasviel/ControlNet
  # @markdown  - https://github.com/Mikubill/sd-webui-controlnet
  # @markdown  - https://huggingface.co/webui/ControlNet-modules-safetensors/

  %cd {tool_dir}/extensions
  !git clone https://github.com/Mikubill/sd-webui-controlnet.git
  # !git clone https://github.com/fkunn1326/openpose-editor.git
  !git clone https://github.com/nonnonstop/sd-webui-3d-open-pose-editor.git

  control_seg = False # @param {type:"boolean"}
  control_openpose = True # @param {type:"boolean"}
  control_canny = False # @param {type:"boolean"}
  control_depth = False # @param {type:"boolean"}
  control_hed = False # @param {type:"boolean"}
  control_mlsd = False # @param {type:"boolean"}
  control_normal = False # @param {type:"boolean"}
  control_scribble = False # @param {type:"boolean"}
  t2iadapter_keypose = False # @param {type:"boolean"}
  t2iadapter_openpose = False # @param {type:"boolean"}
  t2iadapter_seg = False # @param {type:"boolean"}
  t2iadapter_sketch = False # @param {type:"boolean"}

  control_net_models = []
  if t2iadapter_keypose == True:
      control_net_models.append("t2iadapter_keypose")
  if t2iadapter_seg == True:
      control_net_models.append("t2iadapter_seg")
  if t2iadapter_sketch == True:
      control_net_models.append("t2iadapter_sketch")
  if t2iadapter_openpose == True:
      control_net_models.append("t2iadapter_openpose")
  if control_canny == True:
      control_net_models.append("control_canny")
  if control_depth == True:
      control_net_models.append("control_depth")
  if control_hed == True:
      control_net_models.append("control_hed")
  if control_mlsd == True:
      control_net_models.append("control_mlsd")
  if control_normal == True:
      control_net_models.append("control_normal")
  if control_openpose == True:
      control_net_models.append("control_openpose")
  if control_scribble == True:
      control_net_models.append("control_scribble")
  if control_seg == True:
      control_net_models.append("control_seg")

  if Path('sd-webui-controlnet').is_dir():
      %cd sd-webui-controlnet/models
      for control_net_model in control_net_models:
        ! wget -nc --content-disposition https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/{control_net_model}-fp16.safetensors

# Launch
Provide a directory to models (sub dir is optional) and then run this script. You will get something like xxx.gradio.app, click it to go to the deployed Web UI.

---

This script use some parameters to run Web UI. The command line parameters are available at [stable-diffusion-webui Wiki](https://github.com/AUTOMATIC1111/stable-diffusion-webui/wiki/Command-Line-Arguments-and-Settings)

The following parameters are the most important ones for use via colab & gradio:
  - `--share` - create an online gradio.app link
  - `--gradio-debug` - print output to console
  - `--gradio-auth YOUR_ACCOUNT:YOUR_PWD` - create a login account

*   In order to prevent others from crawling your online deployment for use, it is strongly recommended to modify the username and password, and `replace account & password with others`.

---
My Quality Prompt
> 1girl, (close to viewer:1.4), Perfect details,extremely fine and beautiful, Amazing,beautiful background,finely detail,Depth of field,extremely detailed 8k wallpaper,(magnificent), ((best quality)), ((masterpiece)), ((ultra-detailed)), ((an extremely delicate and beautiful),((extremely detailed)), optical, reflective, epic,Hyperdetail, global illumination, epic scenes, beautiful and delicate

> beautiful complex, oil on canvas, (masterpiece:1.3),(best quality,ultra-detailed,illustration:1.2), (extremely detailed cg,8k,high resolution), flawless, clean, supreme detail, highly detailed, sharp focus, professional artwork, famous artwork, cinematic lighting, cinematic bloom

> FOREGROUND, REFLECTION, SACREDNESS, LANDSCAPE, BRIGHT, FHD, 4K, HIGH RESOLUTION, HYPERREALISM, (photorealistic, realistic, RAW photo:1.3), (masterpiece, best quality, ultra highres, ultra-detailed:1.5)

My Negative Prompts:
> (frown:1.5). (text, title, logo, signature:1.5), (worst quality, low quality:1.4), (realistic, lip, nose, tooth, rouge, lipstick, eyeshadow:1.0), (dusty sunbeams:1.0), (abs, muscular, rib:1.0), (depth of field, bokeh, blurry:1.4), (greyscale, monochrome:1.0)

> (lowres, blurry, worst quality, low quality:1.21), normal quality, multiple breasts, (mutated hands and fingers:1.5), (long body:1.3), (mutation, poorly drawn:1.2), black-white, bad anatomy, disfigured, deformed, mutation, mutilated, ugly, poorly drawn face, cloned face, (unclear blurry eyes:1.331), bad eyes, fused eyes, poorly drawn eyes, extra eyes, long neck, bad hands, poorly drawn hands, multiple limbs, extra limbs, malformed limbs, missing arms, missing fingers, interlocked fingers, extra fingers, fused fingers, too many fingers, long nails, missing legs, extra legs, broken legs, bad feet, extra digit, fewer digits, glitchy, (jpeg artifacts, ((signature, watermark, username)), text, error, multiple_views, reference_sheet)

Common Negative Prompts:
> lowres, blurry, worst quality, low quality, normal quality,bad anatomy, disfigured, deformed, mutation, mutilated, ugly, totem pole,poorly drawn face,  cloned face, several faces, long neck, mutated hands, bad hands, poorly drawn hands,extra limbs, malformed limbs, missing arms, missing fingers, extra fingers, fused fingers, too many fingers,missing legs, extra legs, malformed legs, extra digit, fewer digits, glitchy, cropped, jpeg artifacts, signature, watermark, username, text, error

> plain background, poorly drawn face, poorly drawn hands, watermark, censored, (mutated hands and fingers), ugly

In [ ]:
#@title Launch
%cd {tool_dir}

# security
account = "MY_ACCOUNT" #@param {type:"string"}
pwd = "MY_PWD" #@param {type:"string"}

load_model_cmd = f"--ckpt-dir {model_dir}"

if use_specific_model:
  load_model_cmd = f"--ckpt {model}"
else:
  # update the first model to load
  !sed -i "s#THE_FIRST_MODEL.ext#{model_filename}#g" {tool_dir}/config.json

# use param to run tool
# ref: https://github.com/AUTOMATIC1111/stable-diffusion-webui/wiki/Command-Line-Arguments-and-Settings
# --no-progressbar-hiding (may slow down)
# Speed to gen pic: --opt-split-attention OR --xformers
# --no-hashing
# --opt-sdp-no-mem-attention --opt-channelslast
arg = f"--xformers --opt-channelslast --no-half-vae --share --gradio-debug --gradio-auth {account}:{pwd} --disable-safe-unpickle --enable-insecure-extension-access {load_model_cmd}"

!COMMANDLINE_ARGS="{arg}" REQS_FILE="requirements.txt" python launch.py

Streaming output truncated to the last 5000 lines.
 72% 29/40 [00:14<00:05,  2.00it/s]
 75% 30/40 [00:14<00:04,  2.00it/s]
 78% 31/40 [00:15<00:04,  2.00it/s]
 80% 32/40 [00:15<00:04,  2.00it/s]
 82% 33/40 [00:16<00:03,  2.00it/s]
 85% 34/40 [00:16<00:02,  2.00it/s]
 88% 35/40 [00:17<00:02,  2.00it/s]
 90% 36/40 [00:17<00:01,  2.01it/s]
 92% 37/40 [00:18<00:01,  2.01it/s]
 95% 38/40 [00:18<00:00,  2.01it/s]
 98% 39/40 [00:19<00:00,  2.00it/s]
100% 40/40 [00:19<00:00,  2.01it/s]

Total progress: 100% 40/40 [00:21<00:00,  1.85it/s]
  0% 0/40 [00:00<?, ?it/s]
  2% 1/40 [00:00<00:21,  1.82it/s]
  5% 2/40 [00:01<00:19,  1.96it/s]
  8% 3/40 [00:01<00:18,  2.00it/s]
 10% 4/40 [00:02<00:17,  2.03it/s]
 12% 5/40 [00:02<00:17,  2.03it/s]
 15% 6/40 [00:02<00:16,  2.05it/s]
 18% 7/40 [00:03<00:16,  2.04it/s]
 20% 8/40 [00:03<00:15,  2.05it/s]
 22% 9/40 [00:04<00:15,  2.05it/s]
 25% 10/40 [00:04<00:14,  2.06it/s]
 28% 11/40 [00:05<00:14,  2.06it/s]
 30% 12/40 [00:05<00:13,  2.06it/s]
 32% 13/40 [00